# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')

NameError: name 'os' is not defined

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [2]:
import pandas as pd

df = pd.read_csv('onsemi_qa.csv', encoding='utf-8')
df.tail()

,구분,QA
25,일자리,노인 일자리 및 사회활동 지원사업에 대해서 문의하고 싶은데 전화번호가 어떻게 돼? ...
26,의료지원,노인을 위한 치과 치료에 대한 지원 사업이 있어? 네 있습니다. 보건복지부 기초의료...
27,의료지원,"의료급여 틀니, 치과임플란트 지원 사업의 지원 대상자이 어떻게 되나요? 의료급여 틀..."
28,의료지원,"의료급여 틀니, 치과 임플란트 지원 사업 신청 방법은 어떻게 돼? 틀니, 치과임플란..."
29,의료지원,"의료급여 틀니, 치과 임플란트 지원 사업에 대해서 문의하고 싶은데 전화번호가 어떻게..."


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./database", 
                    embedding_function = embeddings  
)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [4]:
database.get()

{'ids': ['d6a0943d-8a66-45a4-ae32-72fc0a95da55'],
 'embeddings': None,
 'metadatas': [{'질문': '안녕'}],
 'documents': ['안녕, 안녕하세요! 무엇을 도와드릴까요?'],
 'uris': None,
 'data': None}

In [5]:
text_list = df['QA'].tolist()
metadata_list = df['구분'].tolist()

metadata = [{'구분': category} for category in metadata_list]

documents = [Document(page_content=text, metadata=meta) for text, meta in zip(text_list, metadata)]

database.add_documents(documents)

['2d766de7-2812-4eda-b9de-2ba154eaac49',
 'bcde4ed3-0265-44d9-8dbc-b6dccf19ae51',
 '11e81639-e8e0-485b-9078-3ab16b9c376f',
 'c34464cc-56ae-4611-a4a6-b839f9ac13ba',
 '57866ecd-097f-49b7-a342-366b36ca7ad6',
 '6b42bd7a-3bc3-453d-8464-7f6ae6093664',
 '33ca5f26-9bfa-4b34-9376-4c4d79c25094',
 '1fa5bfe3-6b08-40ff-87ec-fe3ce39546e5',
 '5e4182ad-3c3c-4728-95db-28352158ad79',
 '47d8cd3f-ba07-4769-a98e-087b5c11671a',
 'd5c518cf-e3e4-40a4-950e-5d9a49973492',
 '9ddfc329-0d0d-4456-92db-b208b24ebcc4',
 'e0e8bcaa-73ea-4f23-8608-a7e4e7b1dcb9',
 '3f1c76dc-53e6-47bf-a019-29fc49d43b83',
 '7b48d4b3-000b-4e24-ad35-a1907734091b',
 '741c39aa-9ae5-4c90-99c8-435bd8712ede',
 '9878dc07-80c6-44a8-8377-905a15efbf73',
 'f68d475e-7338-44d2-90b1-d7f7c7a01dc9',
 '2f673f4f-9c64-4c78-92df-3c6740da7e0a',
 '471989bc-33f0-437e-a190-4ce4c3739e54',
 '1ed9c89c-3f4c-4962-9884-31b2c5891a0e',
 '043ebc8d-4cc4-4d7e-96d6-1d83729c881c',
 '0fac9e84-6f7f-4b74-8282-457ff892fb19',
 '78a1147c-6426-440f-bf65-b1b03e4d4b27',
 '2dff48ae-d143-

* 입력된 데이터 조회

In [6]:
dtt = database.get()
docu = pd.DataFrame(dtt)
docu[['metadatas','documents']]

,metadatas,documents
0,{'구분': '교통지원'},대중교통 지원에 대해서 문의 하고 싶은데 전화 번호가 뭐야? K-패스 고객센터 :...
1,{'구분': '일자리'},노인 일자리 지원 사업이 있어? 네 있습니다. 보건복지부 노인지원과에서 담당하고 있...
2,{'구분': '질문'},등록된 어르신의 정보를 수정하려면 어떻게 해야 하나요? 마이페이지에서 등록된 어르신
3,{'구분': '교통지원'},"대중교통 지원비 신청 방법은 어떻게 돼? 별도 신청 없이 직접 지원합니다. (단, ..."
4,{'구분': '질문'},결제 방법은 어떤 것들이 있나요? 신용카드/체크카드 결제 및 가상계좌 결제가 가능합니다.
5,{'구분': '질문'},서비스 이용 요금은 어떻게 되나요? 무료로 사용 가능합니다.
6,{'구분': '일자리'},노인 일자리 및 사회활동 지원사업의 신청방법은 어떻게 돼? 방법은 2가지가 있습니다...
7,{'구분': '교통지원'},대중교통 지원 해줘? 네 지원합니다. 월 15회 이상 대중교통 이용 비용의 일정비율...
8,{'구분': '질문'},주문한 상품의 배송 상태를 어떻게 확인하나요? 주문 내역에서 확인 가능합니다.
9,{'구분': '질문'},다른 가족 구성원도 계정을 만들 수 있나요? 가족 구성원 당 한 명만 계정 생성 가...


## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

* 모델 선언

* 모델 사용 및 이력 확인

In [18]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
k=3
retriever = database.as_retriever(search_kwargs={"k": k})

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)#대화가 시작될때 memory가 실행되어야 한다.

qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

path = './db_chatlog/chatlog.db'
conn = sqlite3.connect(path)

cursor = conn.cursor()

cursor.execute('''DROP TABLE IF EXISTS history''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

conn.commit()
conn.close()

while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)
    
    dt = datetime.now()
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    score = []
    result2 = database.similarity_search_with_score(query, k = k)
    for doc in result2:
        score.append(round(doc[1], 5))
        
    conn = sqlite3.connect(path)

    history_data = pd.DataFrame({'datetime': [dt],'query':[query],'sim1': [score[0]],'sim2': [score[1]],'sim3': [score[2]],'answer':[result["answer"]]})
    history_data.to_sql('history', conn, if_exists='append', index=False)
    
    d2 = pd.read_sql('SELECT * FROM history', conn)
    display(d2)
    print(memory.load_memory_variables({}))

    conn.close()
lod_df = d2

질문 : 사용 금액은?
--------------------
답변 : 무료로 사용 가능합니다.


,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-27 09:28:19,사용 금액은?,0.22124,0.22124,0.29529,무료로 사용 가능합니다.


{'chat_history': [HumanMessage(content='사용 금액은?'), AIMessage(content='무료로 사용 가능합니다.')]}
질문 : 상태 업데이트 빈도는?
--------------------
답변 : 월 4회입니다.


,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-27 09:28:19,사용 금액은?,0.22124,0.22124,0.29529,무료로 사용 가능합니다.
1,2,2024-06-27 09:28:33,상태 업데이트 빈도는?,0.19596,0.19596,0.36448,월 4회입니다.


{'chat_history': [HumanMessage(content='사용 금액은?'), AIMessage(content='무료로 사용 가능합니다.'), HumanMessage(content='상태 업데이트 빈도는?'), AIMessage(content='월 4회입니다.')]}
질문 : 
--------------------


In [19]:
query = "지원자격은??"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='대중교통 지원 대상이 어떻게 돼? 만 19세 이상 국민 중 월 15회 이상 대중교통 이용자를 대상으로 지원합니다. 외국인 등록번호가 있는 외국인에 한해 회원가입 가능합니다. 외국인 등록번호로 거소사실 여부를 확인할 수 없다면, ""홈페이지""로 가입 가능합니다. ', metadata={'구분': '교통지원'}), 0.291273675140306), (Document(page_content='대중교통 지원 대상이 어떻게 돼? 만 19세 이상 국민 중 월 15회 이상 대중교통 이용자를 대상으로 지원합니다. 외국인 등록번호가 있는 외국인에 한해 회원가입 가능합니다. 외국인 등록번호로 거소사실 여부를 확인할 수 없다면, ""홈페이지""로 가입 가능합니다. ', metadata={'구분': '교통지원'}), 0.291273675140306), (Document(page_content='어르신의 건강 정보 입력은 누가 할 수 있나요? 첫 가입시 보호자님이 작성후 봉사자가 관리합니다.', metadata={'구분': '질문'}), 0.3127854441394578)]
--------------------------------------------------
유사도 점수 : 0.29127, 문서 내용: 대중교통 지원 대상이 어떻게 돼? 만 19세 이상 국민 중 월 15회 이상 대중교통 이용자를 대상으로 지원합니다. 외국인 등록번호가 있는 외국인에 한해 회원가입 가능합니다. 외국인 등록번호로 거소사실 여부를 확인할 수 없다면, ""홈페이지""로 가입 가능합니다. 
유사도 점수 : 0.29127, 문서 내용: 대중교통 지원 대상이 어떻게 돼? 만 19세 이상 국민 중 월 15회 이상 대중교통 이용자를 대상으로 지원합니다. 외국인 등록번호가 있는 외국인에 한해 회원가입 가능합니다. 외국인 등록번호로 거소사실 여부를 확인할 수 없다면, ""홈페이지""로 가입 가능합니다. 
유사도 점수 : 0.31279, 문서 내용: 어르신의 건강 정보 

In [20]:
display(log_df)

NameError: name 'log_df' is not defined

In [ ]:
# def process_query(query, k=3):
#     result = database.similarity_search_with_score(query, k=k)
#     sim1 = round(result[0][1], 5)
#     sim2 = round(result[1][1], 5)
#     sim3 = round(result[2][1], 5)
#     dt = datetime.now()    
#     dt = dt.strftime('%Y-%m-%d %H:%M:%S')
#     answer_result = qa({"question": query})    
#     answer = answer_result["answer"]    
#     return dt, query, sim1, sim2, sim3, answer
# conn = sqlite3.connect(path) 
# while True:    
#     query = input('질문 > ').strip()    
#     if not query:        break   
#     print(f'질문 : {query}')   
#     print('-' * 20)        
#     dt, query, sim1, sim2, sim3, answer = process_query(query)        
#     data = pd.DataFrame({'datetime': [dt],'query': [query],'sim1': [sim1],'sim2': [sim2],'sim3': [sim3],'answer': [answer]})        
#     data.to_sql('history', conn, if_exists='append', index=False)        
#     print(f'답변 : {answer}')    
#     print('=' * 50)
#     df = pd.read_sql('SELECT * FROM history', conn)
#     display(df)
#     conn.close()

질문 : 나는 몇살이야?
--------------------
답변 : 이에 대한 정보는 제가 알 수 없습니다.


,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 15:03:21,kt에이블 스쿨에 지원하고싶은데 어떤 조건들을 만족해야해?,0.23284,0.24214,0.26018,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준으로 재직자는 지원이 ..."
1,2,2024-06-04 15:03:55,내가 만약 취업상태라면 어떻게해야할까?,0.27547,0.29558,0.30972,"KT 에이블스쿨은 미취업자를 대상으로 하고 있어서, 현재 취업 상태일 경우에는 지원..."
2,3,2024-06-04 15:04:27,그러면 에이블스쿨에 학겨하기 전에 퇴사를 하면 참여가능할까?,0.21229,0.25994,0.26083,"KT 에이블스쿨은 미취업자를 대상으로 하고, 교육 시작일 기준으로 재직자는 지원이 ..."
3,4,2024-06-04 16:08:36,나는 26살이야,0.40889,0.42293,0.45180,"네, 해당 교육 과정은 34세 이하의 지원 대상자를 대상으로 합니다. 다만, 35세..."
4,5,2024-06-04 16:09:08,나는 몇살이야?,0.38818,0.39459,0.40453,이에 대한 정보는 제가 알 수 없습니다.


질문 : 나는 26살이야
--------------------


ProgrammingError: Cannot operate on a closed database.